In [2]:
# python
import time
from threading import Thread

from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract
from ibapi.order import Order


# ============== 参数区（请根据需要修改） ==============
HOST = "127.0.0.1"
PORT = 7497            # TWS 模拟账户默认端口：7497；实盘常为 7496
CLIENT_ID = 123        # 任意未占用的客户端ID

quantity = 100000      # 下单数量（GBP基准单位，1标准手=100000）
pip = 0.0001           # GBPUSD 的1 pip
entry_offset_pips = 2  # 入场价相对 last_close 偏移，避免瞬时成交
tp_pips = 100          # 止盈距离
sl_pips = 50           # 止损距离

# 你需要提供下面两个价格（可从你的预测DataFrame/变量中读取）
predicted_close = 1.34500   # 占位：预测的下一日收盘价
last_close = 1.3439        # 占位：最近收盘价/当前价
# ====================================================


def round_fx(price: float, decimals: int = 5) -> float:
    return round(float(price), decimals)


def compute_order_params(predicted: float, last_price: float):
    """
    根据预测方向给出：action(买/卖)、entry/target/stop 三个价格。
    简单规则：
    - 多：在当前价下方少许挂买入限价，止盈+tp，止损-sl
    - 空：在当前价上方少许挂卖出限价，止盈-tp，止损+sl
    """
    if predicted > last_price:
        action = "BUY"
        entry = last_price - entry_offset_pips * pip
        target = entry + tp_pips * pip
        stop = entry - sl_pips * pip
    else:
        action = "SELL"
        entry = last_price + entry_offset_pips * pip
        target = entry - tp_pips * pip
        stop = entry + sl_pips * pip

    # 外汇常见5位小数
    entry = round_fx(entry)
    target = round_fx(target)
    stop = round_fx(stop)
    return action, entry, target, stop


def make_forex_contract(symbol="GBP", currency="USD"):
    c = Contract()
    c.symbol = symbol
    c.secType = "CASH"
    c.exchange = "IDEALPRO"
    c.currency = currency
    return c


def make_bracket(parent_id: int, action: str, qty: float, entry_price: float,
                 take_profit_price: float, stop_loss_price: float):
    """
    生成括号订单（父限价+止盈限价+止损止损），最后一个子单 transmit=True 保证一次性发送。
    """
    is_buy = action.upper() == "BUY"
    exit_action = "SELL" if is_buy else "BUY"

    # Parent LIMIT
    parent = Order()
    parent.orderId = parent_id
    parent.action = action
    parent.orderType = "LMT"
    parent.totalQuantity = qty
    parent.lmtPrice = entry_price
    parent.tif = "GTC"
    parent.transmit = False  # 不立即发送，等最后一个子单发送时一起发送

    # Take Profit LIMIT
    take_profit = Order()
    take_profit.orderId = parent_id + 1
    take_profit.action = exit_action
    take_profit.orderType = "LMT"
    take_profit.totalQuantity = qty
    take_profit.lmtPrice = take_profit_price
    take_profit.tif = "GTC"
    take_profit.parentId = parent_id
    take_profit.transmit = False

    # Stop Loss STP
    stop_loss = Order()
    stop_loss.orderId = parent_id + 2
    stop_loss.action = exit_action
    stop_loss.orderType = "STP"
    stop_loss.totalQuantity = qty
    stop_loss.auxPrice = stop_loss_price  # STP 使用 auxPrice 作为触发价
    stop_loss.tif = "GTC"
    stop_loss.parentId = parent_id
    stop_loss.transmit = True  # 最后一个子单发送时，连同整个括号一起发送

    return parent, take_profit, stop_loss


class IBapi(EWrapper, EClient):
    def __init__(self):
        EClient.__init__(self, self)
        self.nextOrderId = None

    def nextValidId(self, orderId: int):
        super().nextValidId(orderId)
        self.nextOrderId = orderId
        print(f"nextValidId: {orderId}")


    def orderStatus(self, orderId, status, filled, remaining, avgFillPrice, permId,
                    parentId, lastFillPrice, clientId, whyHeld, mktCapPrice):
        print(f"OrderStatus. ID: {orderId}, Status: {status}, Filled: {filled}, "
              f"Remaining: {remaining}, AvgFill: {avgFillPrice}")

    def openOrder(self, orderId, contract, order, orderState):
        print(f"OpenOrder. ID: {orderId}, {contract.symbol}.{contract.currency}, "
              f"{order.action} {order.orderType} @{getattr(order, 'lmtPrice', getattr(order, 'auxPrice', ''))}, "
              f"ParentId: {order.parentId if hasattr(order, 'parentId') else 'NA'}")


def place_bracket_order(predicted: float, last_price: float):
    action, entry, target, stop = compute_order_params(predicted, last_price)
    print(f"方向: {action} | 入场(LMT): {entry} | 止盈(LMT): {target} | 止损(STP): {stop}")

    app = IBapi()
    app.connect(HOST, PORT, clientId=CLIENT_ID)

    # 启动API线程
    api_thread = Thread(target=app.run, daemon=True)
    api_thread.start()

    # 等待 nextOrderId
    for _ in range(20):
        if app.nextOrderId is not None:
            break
        time.sleep(0.5)

    if app.nextOrderId is None:
        print("未获得 nextValidId，停止。")
        app.disconnect()
        return

    contract = make_forex_contract("GBP", "USD")
    parent_id = app.nextOrderId

    parent, tp, sl = make_bracket(
        parent_id=parent_id,
        action=action,
        qty=quantity,
        entry_price=entry,
        take_profit_price=target,
        stop_loss_price=stop
    )

    # 依次下单（注意：只有最后一个子单 transmit=True，会一次性发出整个括号）
    app.placeOrder(parent.orderId, contract, parent)
    app.placeOrder(tp.orderId, contract, tp)
    app.placeOrder(sl.orderId, contract, sl)

    print(f"已提交括号订单，父单ID: {parent_id}")
    # 留一段时间让回调打印
    time.sleep(5)
    app.disconnect()


if __name__ == "__main__":
    place_bracket_order(predicted_close, last_close)


ERROR -1 2104 Market data farm connection is OK:usfuture
ERROR -1 2104 Market data farm connection is OK:usfarm
ERROR -1 2106 HMDS data farm connection is OK:ushmds
ERROR -1 2158 Sec-def data farm connection is OK:secdefil


方向: BUY | 入场(LMT): 1.3437 | 止盈(LMT): 1.3537 | 止损(STP): 1.3387
nextValidId: 1


ERROR 1 10268 The 'EtradeOnly' order attribute is not supported.


已提交括号订单，父单ID: 1


ERROR 2 135 Can't find order with id = 1
ERROR 3 135 Can't find order with id = 1
